In [1]:
import numpy as np
import pandas as pd
import random
import warnings

from paralytics.column_parsing import CategoricalGrouper, ColumnSelector, ColumnProjector, TypeSelector
from paralytics.encoding import TargetEncoder
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier


SEED = 42
random.seed(SEED)
np.random.seed(SEED)

Wczytanie oraz połączenie zbioru treningowego i testowego w celu łatwiejszego dokonywania przekształceń, przy których nie może dojść do wycieku informacyjnego ze zbioru testowego.

In [2]:
df_test = pd.DataFrame(
    {'Pclass': 3, 'Sex': 'male', 'Age': 2, 'SibSp': 3, 'Parch': 1, 'Fare': 10}, 
    index=['kandydat']
)
df_train = pd.read_csv('train.csv')
df = pd.concat([df_train, df_test], sort=True)
df.tail(2)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
890,32.0,NaN,Q,7.75,"Dooley, Mr. Patrick",0,891.0,3,male,0,0.0,370376
kandydat,2.0,NaN,NaN,10.00,NaN,1,NaN,3,male,3,NaN,NaN


In [3]:
df.isnull().sum()

Age            177
Cabin          688
Embarked         3
Fare             0
Name             1
Parch            0
PassengerId      1
Pclass           0
Sex              0
SibSp            0
Survived         1
Ticket           1
dtype: int64

Kabina nie została podana w danych do zadania, tak samo, jak i port, z którego wypłynął statek, stąd skupimy się tylko na pozbyciu się nulli z kolumny: `Age`.

Utworzenie zmiennej pomocniczej: tytuł w celu imputacji nulli `Age` medianami w poszczególnych grupach. Wartym odnotowania jest, iż na czas wypadku praktykowane były następujące tytuły:

* Master: young boys
* Miss: young women
* Mrs: married women
* Ms: women regardless of maritial status
* Mr: men regardless of maritial status

Dodatkowo stworzymy zmienną mówiącą o rozmiarze rodziny, jak i o tym, czy konkretna osoba podróżowała samotnie, nie musząc martwić się o bliskich prawdopodobnym jest, iż osoba mogła sprawniej przemieścić się do kajut ratunkowych.

In [4]:
def create_title(cell):
    if type(cell) == str:
        return cell[cell.find(',') + 2:cell.find('.')]
    else:
        return cell
    
df = df.assign(
    Title=df.Name.apply(create_title)
)

title_translator = {
    'Mr': ['Major', 'Capt', 'Sir', 'Don'], 
    'Mrs': ['Lady', 'the Countess'],
    'Miss': ['Mlle', 'Mme', 'Ms'], 
    'Other': ['Rev', 'Col', 'Jonkheer']
}

for key, value in title_translator.items():
    df.Title.replace(
        to_replace=value,
        value=key,
        inplace=True
    )

# Oddzielnie klasyfikujemy Doktora, ponieważ pojawia się zarówno wśród kobiet, jak i mężczyzn
df.loc[df.Title == 'Dr', 'Title'] = np.where(df[df.Title == 'Dr'].Sex == 'male', 'Mr', 'Mrs')

# Zastępujemy NaNy pośród zmiennej Age medianami w każdej z grup.
# Średnią wyliczamy bez udziału zbioru testowego (u nas ostatni rekord)
# ponieważ dawałoby to wyciek informacji do zbioru treningowego.
age_median_dict = round(
    df.iloc[:-1, :].groupby('Title').Age.agg('median'), 0
).astype(float).to_dict()
df.Age.fillna(df.Title.map(age_median_dict), inplace=True)

df['FamilySize'] = df['SibSp'] + df['Parch']

df['IsAlone'] = 0
df.loc[df['FamilySize'] == 0, 'IsAlone'] = 1

print(
    'Sprawdzenie, czy wszystkie braki danych pośród zmiennej ' 
    '"Age" zostały zaimputowane.'
)
df.isnull().sum()

Sprawdzenie, czy wszystkie braki danych pośród zmiennej "Age" zostały zaimputowane.


Age              0
Cabin          688
Embarked         3
Fare             0
Name             1
Parch            0
PassengerId      1
Pclass           0
Sex              0
SibSp            0
Survived         1
Ticket           1
Title            1
FamilySize       0
IsAlone          0
dtype: int64

Utworzenie rurociągu, który dokonywać będzie następujących przekształceń:

1. Wybranie jedynie podanych zmiennych.
2. Zrzutowanie typów odpowiednio:
    * typy numeryczne na float64,
    * typy kategoryczne na category,
    * typy binarne na bool.
3. Zgrupowanie rzadkich kategorii w jedną kategorię o nazwie: `Others`.
4. Zakodowanie zmiennych kategorycznych na numeryczne przy użyciu estymatora `TargetEncoder`.

In [5]:
df.tail()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,FamilySize,IsAlone
887,19.0,B42,S,30.00,"Graham, Miss. Margaret Edith",0,888.0,1,female,0,1.0,112053,Miss,0,1
888,22.0,NaN,S,23.45,"Johnston, Miss. Catherine Helen ""Carrie""",2,889.0,3,female,1,0.0,W./C. 6607,Miss,3,0
889,26.0,C148,C,30.00,"Behr, Mr. Karl Howell",0,890.0,1,male,0,1.0,111369,Mr,0,1
890,32.0,NaN,Q,7.75,"Dooley, Mr. Patrick",0,891.0,3,male,0,0.0,370376,Mr,0,1
kandydat,2.0,NaN,NaN,10.00,NaN,1,NaN,3,male,3,NaN,NaN,NaN,4,0


In [6]:
df_train = df.iloc[:-1, :].copy()
df_test = df.iloc[-1:, :].copy()

y_train = df_train.pop('Survived')
X_train = df_train

df_test.pop('Survived')
X_test = df_test

# Podanie zmiennych na pozór numerycznych, będących zmiennymi kategorycznymi
manual_projection = {
    'category': ['Pclass']
}

xgboost_model_pipeline = make_pipeline(
    ColumnSelector(columns=[
        'Age', 'Fare', 'Parch', 'Pclass', 'Sex', 
        'SibSp', 'FamilySize', 'IsAlone'
    ]),
    ColumnProjector(manual_projection=manual_projection),
    CategoricalGrouper(),
    FeatureUnion(transformer_list=[
        ('numeric_features', make_pipeline(
            TypeSelector('number'))
        ),
        ('boolean_features', TypeSelector(bool)),
        ('categorical_features', make_pipeline(
            TypeSelector('category'),
            TargetEncoder(cv=5, inner_cv=3, alpha=5, random_state=SEED),
            StandardScaler())
        )
    ]),
    XGBClassifier()
)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    xgboost_model_pipeline.fit(X_train, y_train)
    
survival_proba = xgboost_model_pipeline.predict_proba(pd.DataFrame(X_test))[:,1]

print(
    'Prawdopodobieństwo, że wskazana osoba przeżyje wynosi: '
    '{0:.1f}%'.format(100 * survival_proba[0])
)

Prawdopodobieństwo, że wskazana osoba przeżyje wynosi: 34.5%
